# FAAH Genotype Analysis

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
from statannot import add_stat_annotation
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.factorplots import interaction_plot
import sklearn 
from scipy.stats import ttest_ind
from datetime import date
today = str(date.today())

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [ ]:
print(sm.__version__)
print(sklearn.__version__)
print(bct.__version__)

In [ ]:
#Read in Rest data
home = '<path to data>'
rest_ids_raw = pd.read_csv(home + '/rest_subjectlist.csv', index_col=0) #All ids with resting state scans

In [ ]:
# Read in list of subjects who passed exclusion criteria
included = pd.read_csv(home + '/symptoms/SubjectsIncluded_FromScreener.csv', 
                       header = None, names = ['subjectkey'])
included['subjectkey'] =included['subjectkey'].str.replace('sub-NDAR', 'NDAR_')

#Drop incorrect IDs per Known Issues
gen_id_bad = pd.read_csv(home + '/genetics3.0/111.incorrect.genetic.subjectid.csv',
                        dtype = 'str').rename(columns = {'x':'subjectkey'})

# Merge with subjects who have neuroimaging data to create final subject list
rest_ids_init = pd.merge(rest_ids_raw, included, how = 'inner')
print(rest_ids_init.shape)

In [ ]:
# Read in batch infor for genotyping--need to exclude plate 461
plate = pd.read_csv(home + '/genetics3.0/ABCD_release3.0_.batch_info.txt', sep = '\t')
badplate = plate[plate['Axiom_Plate']== 461]
badplate_ids = pd.Series(badplate['abcd.id_redcap'], name = 'subjectkey')

In [ ]:
#Merge bad ids together
print(len(badplate_ids), len(gen_id_bad))
all_badids = pd.concat([badplate_ids, gen_id_bad])
print(len(all_badids))

In [ ]:
def drop_bad_rows(all_subs, bad_subs):
    df = all_subs.set_index('subjectkey')
    for i in range(0, len(bad_subs)):
        try:
            bsub = bad_subs['subjectkey'][i]
            df.drop(bsub, axis=0, inplace = True)
        except:
            pass
        
    return df.reset_index()
        
rest_ids = drop_bad_rows(rest_ids_init, all_badids)
print(rest_ids.shape)

In [ ]:
#Read in behavioral covariates
behav = pd.read_csv(home + '/symptoms/full_sample_behav.csv', header = 0).drop('pds', axis=1).drop('eventname', axis=1)

#Read in genetics data
gene_dat = pd.read_csv(home + '/genetics3.0/rs324420-rw.txt', sep='\t', names = ['subjectkey', 'gene',1, 2, 3])

In [ ]:
#Read in youth mental health scores
ymh = pd.read_csv(home + '/symptoms/abcd_mhy02.txt', sep = '\t', engine = 'python')
ymh1 = ymh.drop(0, axis = 0)
#ymh_bl = ymh[ymh['eventname'] == 'baseline_year_1_arm_1']
ymh1['cbcl_eventname'] = ymh1['eventname'].str.replace('1_year_', '').str.replace('2_year_', 'twoyear')
bis_scores = ymh1[["subjectkey", "cbcl_eventname", "bis_y_ss_bis_sum"]]

In [ ]:
#Read in genetic ancestry
anc = pd.read_csv(home + '/genetics3.0/acspsw03.txt', header = 0, sep = '\t', 
                   engine = 'python').drop(0, axis=0)
anc1 = anc[anc.eventname == 'baseline_year_1_arm_1']
g_anc = anc1[["subjectkey", "genetic_af_african", "genetic_af_european", "genetic_af_east_asian", "genetic_af_american"]]
g_anc[["genetic_af_african", "genetic_af_european", "genetic_af_east_asian", "genetic_af_american"]] = g_anc[["genetic_af_african", "genetic_af_european", "genetic_af_east_asian", "genetic_af_american"]].astype('float')

In [ ]:
genotype = []
for i in range(0, len(gene_dat)):
    if gene_dat['gene'][i] == '0|0':
        genotype.append(1)
    elif gene_dat['gene'][i] == '0|1':
        genotype.append(2)
    elif gene_dat['gene'][i] == '1|0':
        genotype.append(2)
    elif gene_dat['gene'][i] == '1|1':
        genotype.append(2)
    else:
        genotype.append(np.nan)

# Add genotype factor variable to genetics data
gene_dat['genotype'] = pd.Series(genotype, dtype='category')

In [ ]:
# Read in longitudinal CBCL data 
cbcl1 = pd.read_csv(home + '/symptoms/abcd_cbcls01.txt', header = 0, sep = '\t', 
                   engine = 'python').drop(0, axis=0) #Drop descriptive line
cbcl=cbcl1
cbcl['subjectkey'] = cbcl['src_subject_id']

#Zscore/modify CBCL variables of interest
cbcl['cbcl_int_r'] = zscore(cbcl['cbcl_scr_syn_internal_r'].astype(float), nan_policy='omit')
cbcl['cbcl_ext_r'] = zscore(cbcl['cbcl_scr_syn_external_r'].astype(float), nan_policy='omit')
cbcl['cbcl_tp_r'] = zscore(cbcl['cbcl_scr_syn_totprob_r'].astype(float), nan_policy='omit')
cbcl['cbcl_anxdisord_r'] = zscore(cbcl['cbcl_scr_dsm5_anxdisord_r'].astype(float), nan_policy='omit')
cbcl['cbcl_anxdisord_t'] = zscore(cbcl['cbcl_scr_dsm5_anxdisord_t'].astype(float), nan_policy='omit')
cbcl['cbcl_anxdep_r'] = zscore(cbcl['cbcl_scr_syn_anxdep_r'].astype(float), nan_policy='omit')
cbcl['cbcl_anxdep_t'] = zscore(cbcl['cbcl_scr_syn_anxdep_t'].astype(float), nan_policy='omit')
cbcl['cbcl_age'] = cbcl['interview_age'].astype(float)
cbcl['cbcl_eventname'] = cbcl['eventname'].str.replace('1_year_', '').str.replace('2_year_', 'twoyear')
#cbcl = cbcl[cbcl['eventname'] != '2_year_follow_up_y_arm_1']
cbcl = cbcl.drop(['sex'], axis=1)

In [ ]:
#Read in ethnicity
eth = pd.read_csv(home + '/ethnicity.csv', header = 0, engine = 'python')

In [ ]:
# Read in Pubertal Development Scores
pds1 = pd.read_csv(home + '/symptoms/abcd_ssphy01.txt', header = 0, sep = '\t', 
                   engine = 'python').drop(0, axis=0) #Drop descriptive line

#Confirm no 0s in baseline data
assert 0.0 not in pds1['pds_y_ss_male_category']
assert 0.0 not in pds1['pds_y_ss_female_category']

#Drop longitudinal data
pds=pds1[pds1.eventname == 'baseline_year_1_arm_1'].replace(np.nan, 0.0)
pds['pds'] = pds['pds_y_ss_male_category'].astype(float) + pds['pds_y_ss_female_category'].astype(float)
pds = pds[['subjectkey', 'eventname', 'pds']].replace(0.0, np.nan)

In [ ]:
ksads_df = ksads_fu
#Create symptom score sum -- select relevant cols
anx_cols = ['subjectkey', 'cbcl_eventname', 'ksads_8_301_t', 'ksads_8_302_t',
           'ksads_8_303_t', 'ksads_8_304_t', 'ksads_8_307_t', 'ksads_8_308_t',
           'ksads_8_309_t', 'ksads_8_310_t', 'ksads_8_311_t', 'ksads_8_312_t',
           'ksads_8_313a_t', 'ksads_8_313b_t', 'ksads_10_320_t', 'ksads_10_321_t',
           'ksads_10_322_t', 'ksads_10_323_t', 'ksads_10_324_t', 'ksads_10_325_t',
           'ksads_10_326_t', 'ksads_10_327_t', 'ksads_10_328_t', 'ksads_10_329_t',
           'ksads_10_330_t']
anx_df = ksads_df[anx_cols].replace(np.nan, 0.0)
sum_scores = anx_df.loc[:,'ksads_8_301_t':'ksads_10_330_t'].astype(float).sum(axis=1)
anx_df['anx_sum'] = sum_scores
anx_df_final = anx_df[anx_df['anx_sum'] > 0][["subjectkey", "cbcl_eventname", "anx_sum"]]

In [ ]:
#Merge final data
#One timepoint
m1 = pd.merge(gene_dat[['subjectkey', 'genotype']], behav, on='subjectkey', how='inner')
m2 = pd.merge(m1, rest_ids, on='subjectkey', how='inner')
m3 = pd.merge(m2, g_anc, on='subjectkey', how='inner')
m4  = pd.merge(m3, eth, on = 'subjectkey', how = 'left')
m5 = pd.merge(m4, pds, on = 'subjectkey', how = 'inner')
#Two timepoints
m6 = pd.merge(m5, cbcl, on = 'subjectkey', how='inner')
m7 = pd.merge(m6, bis_scores, on = ['subjectkey', 'cbcl_eventname'], how = 'inner')
#Master dataframe
master = m6
print(master.shape)

ksads_df = pd.merge(m5, anx_df_final, on = 'subjectkey', how = 'inner')
#Save CSVs of interest
# baseline.to_csv(home + '/ABCD_FAAH_RestingStateSample_{}.csv'.format(today))

### Examine interaction between genotype and timepoint in predicting CBCL Anxiety scores

In [ ]:
#Select only vars of interest
model_vars = master[["subjectkey", 'genotype', 'cbcl_tp_r', 'cbcl_int_r', 'cbcl_ext_r', 'cbcl_anxdisord_r', 'cbcl_scr_dsm5_anxdisord_r',
                     'cbcl_anxdep_r', "cbcl_eventname", "cbcl_age", "pds", "sex", "rel_family_id", "site_id_l", #"bis_y_ss_bis_cbcl_scr_syn_anxdep_rsum",
                     "mri_info_deviceserialnumber", 'genetic_af_african', 'genetic_af_european', "fam_income", "mat_ed", 'demo_ethn_v2',
                     'genetic_af_east_asian', 'genetic_af_american', 'mri_info_manufacturer']] 
model_vars['genotype'] = model_vars['genotype'].astype('category')
model_vars['cbcl_scr_dsm5_anxdisord_r'] = model_vars['cbcl_scr_dsm5_anxdisord_r'].astype(float)
selected = model_vars.dropna(how='any', axis=0) #Clean up any NaNs in selected data
selected['cbcl_eventname'] = selected['cbcl_eventname'].str.replace('1_year_follow_up_y_arm_1', 'follow_up_y_arm_1').astype('category')
selected['Genotype'] = selected['genotype'].replace(1, "CC").replace(2, 'AA/AC')
selected['fam_income'] = selected['fam_income'].astype(float).replace(777, np.nan).replace(999, np.nan).astype('category') #Recode missing values
selected['mat_ed'] = selected['mat_ed'].astype(float).replace(777, np.nan).replace(999, np.nan).astype('category') #Recode missing values
selected['demo_ethn_v2'] = selected['demo_ethn_v2'].astype(float).replace(777, np.nan).replace(999, np.nan).astype('category') 

#Filter out subs excluded due to bad plate
selected=selected[selected.subjectkey != 'NDAR_INVV7NEVHLK'] #Remove bad genotyping subscbcl_scr_syn_anxdep_r
selected=selected[selected.subjectkey != 'NDAR_INVA7RNTEHU'] #Remove bad genotyping subs
selected=selected.reset_index() #Avoid in|dexing errors in plots
print(selected.shape) #Final shape

#Filter by timepoint
baseline = selected[selected['cbcl_eventname'] == 'baseline_year_1_arm_1'].dropna().reset_index()
print(baseline.shape)
followup = selected[selected['cbcl_eventname'] == 'twoyearfollow_up_y_arm_1'].dropna().reset_index()

#Create wide follow-up data
# followup['bis_fu'] = followup['bis_y_ss_bis_sum']
followup['cbcl_anxdisord_fu'] = followup['cbcl_anxdisord_r']
print(followup.shape)

#Seleted plus followup scores in wide format
selected_plus = pd.merge(baseline, followup[["subjectkey", 'cbcl_anxdisord_fu']], how = 'inner').dropna(how = 'any', axis=0)
print(selected_plus.shape)

#Assertion sanity checks
assert len(baseline) == 3109 ## For CBCL, ethnicity no NAs
assert np.nan not in baseline['sex']
assert np.nan not in baseline['fam_income']
assert np.nan not in baseline['mat_ed']
assert np.nan not in baseline['cbcl_age']

#Subs with both baseline and follow up data
long_subs = selected[selected['cbcl_eventname'] == 'twoyearfollow_up_y_arm_1'].dropna()['subjectkey']
long_select = pd.merge(selected, long_subs, how='inner')

# #Write out CSVs of interest
# selected_plus.to_csv(home + '/Selected_Masterfile_twoTPs_{}.csv'.format(today))
# baseline.to_csv(home + '/Baseline_Masterfile_n={}_{}.csv'.format(len(baseline),today))
# long_select.to_csv(home + '/symptoms/cbclcheck_longitudinalsubs.csv')

In [ ]:
# Demogs
print('Mean Age: {}'.format(baseline['cbcl_age'].mean()/12))
print('Min Age: {}'.format(baseline['cbcl_age'].min()/12))
print('Max Age: {}'.format(baseline['cbcl_age'].max()/12))
print('Std Dev of Age: {}'.format(baseline['cbcl_age'].std()/12))
print('Prop. Female: {}'.format(len(baseline[baseline['sex']== 2])/len(baseline['sex'])))
print('Prop. Male: {}'.format(len(baseline[baseline['sex']== 1])/len(baseline['sex'])))

In [ ]:
#Set model variables
yvar = 'cbcl_scr_dsm5_anxdisord_r'

### Examine associations between CBCL Anxiety scores genotype at baseline

In [ ]:
#Model whether CBCL scores are predicted by genotype at baseline
data = baseline
data[yvar] = data[yvar].astype(float)
#Actual model
mod1 = sm.MixedLM.from_formula("{} ~ genotype + cbcl_age + sex + pds + fam_income + mat_ed + demo_ethn_v2 + genetic_af_african + genetic_af_european + genetic_af_east_asian + genetic_af_american".format(yvar), 
                                 re_formula="1", vc_formula={"fam_id": "0 + C(rel_family_id)",
                                                             "subjectkey": "0 + C(subjectkey)"},
                groups="site_id_l", data=data )
#results = mod1.fit()
print('T-value: {}'.format(pd.DataFrame(results.tvalues).loc['genotype[T.2]']))
summary = results.summary()
print(summary)

In [ ]:
#Plot non-signficant differences between genotypes at baseline

sns.boxplot(x="Genotype", y=yvar, palette = ['#e75454', '#5c78ec'], data=baseline)
plt.title('Group Differences in Anxiety Symptoms by Genotype')
plt.ylabel('CBCL Anxiety Disorder Scores')

In [ ]:
#Histograms of anxiety symptom distribution
sns.histplot(data = baseline, x = yvar, hue = 'Genotype', palette = ['#e75454', '#5c78ec'])
plt.xlabel('CBCL Anxiety Disorder Scores')

### Network Based Statistic Analysis

In [ ]:
import bct
from glob import glob
from nilearn import connectome as conn
from nilearn import plotting
from datetime import date
import matplotlib.cm as cm
today = str(date.today())

In [ ]:
#Set paths
all_mats_dir = '/Users/lucindasisk/Box/LS_Folders/CANDLab/Projects/ABCD_FAAH/rest_matrices_averaged'
all_behav_dir = all_mats_dir

#Glob list of all files in this dir that match string
subfiles = glob(all_mats_dir + '/sub-NDAR*.csv')

#Create list of subjectids
subs = []
for i in range(0, len(subfiles)):
    s = subfiles[i]
    x = s.replace(all_mats_dir + '/', '')
    sub = x.split('_')[0]
    subs.append(sub)

#Rename subjectkey 
subsers = pd.Series(subs, name = 'subjectkey').str.replace('sub-NDAR', 'NDAR_')

#Merge with 'selected' dataframe (thus excluding subs that did not pass QC) 
data1 = pd.merge(subsers, baseline, how = 'inner', on='subjectkey')
data = data1[data1.cbcl_eventname  == 'baseline_year_1_arm_1'] #'''follow_up_y_arm_1'] #

assert len(data) == len(baseline.dropna())

type1 = data[data.genotype == 1].drop('level_0', axis=1).reset_index()
type2 = data[data.genotype == 2].drop('level_0', axis=1).reset_index()

In [ ]:
#Create empty matrices to append data into
mat_1 = np.zeros(shape=(368, 368, len(type1)))
mat_2 = np.zeros(shape=(368, 368, len(type2)))

assert mat_1.shape[2] + mat_2.shape[2] == len(baseline)

# Read in data for type 1
for x in range(0, len(type1)):
    #Set subject
    sub = type1['subjectkey'][x].replace('NDAR_',  'sub-NDAR')
    try:
        # Read in data and append in order (3rd dim)
        print("Working on {}, {} out of {}".format(sub, x, len(type1)))
        rest_file = glob(all_mats_dir + '/{}*.csv'.format(sub))[0]
        mat = pd.read_csv(rest_file, header=0, index_col=0, engine='python').to_numpy()[:,0:368]
        assert True not in np.isnan(mat)
        mat_1[:,:,x] = mat
    except:
        print("Error on {}".format(sub))

# Read in data for type 2
for x in range(0, len(type2)):
    #Set subject
    sub = type2['subjectkey'][x].replace('NDAR_',  'sub-NDAR')
    try:
        # Read in data and append in order (3rd dim)
        print("Working on {}, {} out of {}".format(sub, x, len(type2)))
        rest_file = glob(all_mats_dir + '/{}*.csv'.format(sub))[0]
        mat = pd.read_csv(rest_file, header=0, index_col=0, engine='python').to_numpy()[:,0:368]
        assert True not in np.isnan(mat)
        mat_2[:,:,x] = mat
    except:
        print("Error on {}".format(sub))

In [ ]:
# #Run Network-Based Statistic! 
# k = 10000
# thresh = 1.96
# nbs_pval, nbs_adj, nbs_null = bct.nbs_bct(x = mat_1, y = mat_2, 
#                                           thresh = thresh, k=k, #Thresholded at t= 1.96, p=0.05 (t= 2.58; p=0.01) according to calculation here (df 2128; for two-tailed) https://statscalculator.com/tcriticalvaluecalculator?x1=0.01&x2=2128
#                                           tail='both', paired=False, verbose=False)

In [ ]:
# print(nbs_pval)

In [ ]:
## Save out computed adjacency matrix
today = str(date.today())

#Read in adjacency matrix generated from cross validation
nbs_adj = pd.read_csv(home + '/Final_Adjacency_Matrix_nRuns19936_2021-03-03.csv',header = 0, index_col = 0, dtype = int).to_numpy()
print(nbs_adj.shape)
network_value = 1.0

In [ ]:
#Read in Shen Atlas information for plotting
shen_index = pd.read_csv(home + '/shen368/shen_368_BAindexing_MNI', header = 0, names = ['subunit', 'name', 'MNI_X', 'MNI_Y', 'MNI_Z', 'Tal_X', 'Tal_Y', 'Tal_Z'])
shen_small = shen_index.sort_values(by='subunit')
#Select MNI Coordinates
shen_select = shen_small[['MNI_X', 'MNI_Y', 'MNI_Z']].to_numpy()

In [ ]:
# #Plot in glass brain
# from nilearn.connectome import ConnectivityMeasure
# nbs_adj_edited = pd.DataFrame(nbs_adj).replace([11.0, 5.0, 7.0, 2.0, 3.0, 4.0, 8.0, 6.0, 9.0, 10.0], 0.0)

# plotting.plot_connectome(adjacency_matrix = nbs_adj_edited,
#                          node_coords = shen_select,
#                          node_size = 6)

In [ ]:
#Extract mask for only significant network (filter)
def get_mask(adj_mat, val):
    mask = np.argwhere(adj_mat == val)
    print(mask.shape)
    return mask

all_network_mask = get_mask(nbs_adj, network_value)

In [ ]:
# Filter mask to remove duplicate indices
def filter_mask(mask):
    mask = mask.tolist()
    final_mask = []
    for i in range(0, len(mask)):
        pair = mask[i]
        v1 = pair[0]
        v2 = pair[1]
        if [v2, v1]  in final_mask:
            pass
        else:
            final_mask.append([v1, v2])
    print("Original mask length: {}".format(len(mask)))
    print("Length with duplicates removed: {}".format(len(final_mask)))
    return final_mask
    
network_mask = filter_mask(all_network_mask)   

In [ ]:
# Create new adjacency matrix

def get_adj_mat(mask):
    final_mat = np.zeros([368, 368])
    for j in range(0, len(mask)):
        val1 = mask[j][0]
        val2 = mask[j][1]
        final_mat[val1, val2] = 1.0
        final_mat[val2, val1] = 1.0
    return final_mat

In [ ]:
#Get list of subject IDS for diff genotypes
t1subs = type1['subjectkey']
t2subs = type2['subjectkey']

#Function to extract values
def get_network_values(mask, mats, subs, valence):
    n_pairs = int(len(mask))
    mask_mat = np.ones((mats.shape[2], n_pairs))
    for i in range(0, n_pairs):
        inds = mask[i]
        x = inds[0]
        y = inds[1]
        mask_mat[:, i]= mats[x, y]
    net_df = pd.DataFrame(mask_mat)
    net_df_mean = net_df.sum(axis=1)
    net_s = pd.Series(net_df_mean, name = "net_{}_var".format(valence))
    return net_s, net_df

#Function to separate left and right tails
def get_tails(mask, ctl_mat, pt_mat):
    n_pairs = int(len(mask))
    positive_edges = []
    negative_edges = []
    for i in range(0, len(mask)):
        inds = mask[i]
        x = inds[0]
        y = inds[1]
        col1= ctl_mat[x, y]
        col2= pt_mat[x, y]
        t, p = ttest_ind(col1, col2)
        if t > 0:
            #print('{} > than {}'.format(col1.mean().round(3), col2.mean().round(3))) (Sanity check)
            negative_edges.append(inds)
        if t < 0:
            #print('{} < than {}'.format(col1.mean().round(3), col2.mean().round(3))) (Sanity check)
            positive_edges.append(inds)
    return positive_edges, negative_edges
            

In [ ]:
#Filter into positive and negative tails
pos_tail, neg_tail = get_tails(network_mask, mat_1, mat_2)
print(len(pos_tail), len(neg_tail))

#Get positive and negative tails as dataframes for subs
pos_network_t1, pos_network_t1_df = get_network_values(pos_tail, mat_1, t1subs, 'pos')
pos_network_t2, pos_network_t2_df = get_network_values(pos_tail, mat_2, t2subs, 'pos')
print(pos_network_t1.shape, print(pos_network_t2.shape))

neg_network_t1, neg_network_t1_df = get_network_values(neg_tail, mat_1, t1subs, 'neg')
neg_network_t2, neg_network_t2_df = get_network_values(neg_tail, mat_2, t2subs, 'neg')
print(neg_network_t1.shape, print(neg_network_t2.shape))

In [ ]:
# Create adjascency matrices for pos and neg edges
pos_mattt = get_adj_mat(pos_tail)
neg_mattt = get_adj_mat(neg_tail)
pd.DataFrame(pos_mattt).to_csv(home + '/Final_PosEdges_Adjacency_Matrix_{}.csv'.format(today), index = False, header = False)
pd.DataFrame(neg_mattt).to_csv(home + '/Final_NegEdges_Adjacency_Matrix_{}.csv'.format(today), index=False, header=False)

In [ ]:
# Concatenate networks into one dataframe
t1_nets = pd.concat([t1subs, pos_network_t1, neg_network_t1], axis=1)
t2_nets = pd.concat([t2subs, pos_network_t2, neg_network_t2], axis=1)
all_nets = pd.concat([t1_nets, t2_nets], axis=0)
all_nets['subjectkey'] = all_nets['subjectkey'].str.replace('sub-NDAR', 'NDAR_')

#Full_dfs 
t1_dfs = pd.concat([t1subs, pos_network_t1, neg_network_t1_df], axis=1)
t2_dfs = pd.concat([t2subs, pos_network_t2, neg_network_t2_df], axis=1)

net_all_dff = pd.concat([t1_dfs, t2_dfs], axis=0)

In [ ]:
#Merge with Selected DF 
all_df = pd.merge(baseline, all_nets, on = 'subjectkey', how = 'inner')
all_df['genotype'] = all_df['genotype'].astype('category')

#Filter by timepoint
baseline_netdf = all_df[all_df['cbcl_eventname'] == 'baseline_year_1_arm_1'].drop(['index'], axis=1).reset_index()
followup_netdf = all_df[all_df['cbcl_eventname'] == 'follow_up_y_arm_1'].drop(['index'], axis=1).reset_index()

baseline_netdf['demo_ethn_v2'] = baseline_netdf['demo_ethn_v2'].astype('category')
baseline_netdf['fam_income'] = baseline_netdf['fam_income'].astype('category')
baseline_netdf['mat_ed'] = baseline_netdf['mat_ed'].astype('category')

# assert len(baseline_netdf) == 2918 ## for BIS
#assert len(baseline_netdf) == len(selected_plus) # for CBCL

#Save CSVs of interest
#all_df.to_csv(home + '/Selected_NetworkDF_{}.csv'.format(today))

In [ ]:
#Histograms of network summary score distribution --positive
ax = sns.histplot(data = all_df, x = 'net_pos_var', hue = 'Genotype', palette = ['#e75454', '#5c78ec'])
plt.xlabel('Network Summary Statistic (A-allele carriers > CC)', fontsize=14)
plt.ylabel('Count',  fontsize=14)
plt.savefig(home + '/Drafts/Figures/pos_network_histogram.png', dpi=400)

In [ ]:
#Histograms of network summary score distribution --positive
ax2 = sns.histplot(data = all_df, x = 'net_neg_var', hue = 'Genotype', palette = ['#e75454', '#5c78ec'])
plt.xlabel('Network Summary Statistic (A-allele carriers < CC)', fontsize=14)
plt.ylabel('Count',  fontsize=14)
plt.savefig(home + '/Drafts/Figures/neg_network_histogram.png', dpi=400)


In [ ]:
net = 'net_pos_var'
tail = pos_tail

In [ ]:
data = baseline_netdf
#Confirm genotype predicts network conn ***REPLICATED in R (pvals not identical but close)***
data['sex'] = data['sex'].astype('category')
data['sex'].cat.remove_unused_categories(inplace = True)

mod1 = sm.MixedLM.from_formula("{} ~ genotype + cbcl_age + sex +pds + fam_income + mat_ed + demo_ethn_v2 + genetic_af_african + genetic_af_european + genetic_af_east_asian + genetic_af_american".format(net), 
                                 re_formula="1", vc_formula={"fam_id": "0 + C(rel_family_id)",
                                                             "subjectkey": "0 + C(subjectkey)"},
                groups="mri_info_deviceserialnumber", data=data)
results = mod1.fit()
print(' ')
print('T-value: {}'.format(pd.DataFrame(results.tvalues).loc['genotype[T.2]']))
print(' ')
summary = results.summary()
print(summary)

In [ ]:
x = sns.boxplot(x="Genotype", y=net, palette = ['#e75454', '#5c78ec'], data=baseline_netdf)
plt.title('Group Differences in Connectivity by Genotype')

In [ ]:
#Model whether network 7 predicts cbcl ***REPLICATED in R (pvals not identical but close)***
data = baseline_netdf.dropna()
data[yvar] = data[yvar].astype(float)
data['sex'] = data['sex'].astype('category')
data['sex'].cat.remove_unused_categories(inplace = True)
 # + fam_income + mat_ed + demo_ethn_v2
netmod_2 = sm.MixedLM.from_formula("{} ~ {}  + pds + cbcl_age + sex +  fam_income + mat_ed + demo_ethn_v2 + genetic_af_african + genetic_af_european + genetic_af_east_asian + genetic_af_american".format(yvar, net), 
                                 re_formula="1", vc_formula={"fam_id": "0 + C(rel_family_id)",
                                                             "subjectkey": "0 + C(subjectkey)"},
                groups="mri_info_deviceserialnumber", data= data)
results = netmod_2.fit()
print(' ')
print('T-value: {}'.format(pd.DataFrame(results.tvalues).loc['{}'.format(net)]))
print(' ')
summary = results.summary()
print(summary)

In [ ]:
from scipy.stats import pearsonr
corr = pearsonr(baseline_netdf['{}'.format(net)], baseline_netdf[yvar])
print("Whole Sample (size {}): r={}, p={}".format(len(baseline_netdf[yvar]),  round(corr[0], 4), round(corr[1], 4)))

g1 = baseline_netdf[baseline_netdf.genotype == 1.0]
g2 = baseline_netdf[baseline_netdf.genotype == 2.0]

g1corr = pearsonr(g1['{}'.format(net)], g1[yvar])
print("Genotype 1 (size {}):r={}, p={}".format(len(g1), round(g1corr[0], 4), round(g1corr[1], 4)))

g2corr = pearsonr(g2['{}'.format(net)], g2[yvar])
print("Genotype 2 (size {}): r={}, p={}".format(len(g2), round(g2corr[0], 4), round(g2corr[1], 4)))


In [ ]:
data_both = baseline_netdf
data_both[yvar] = data_both[yvar].astype(float)
both_mod = sm.MixedLM.from_formula("{}~ {}*genotype + pds + sex +cbcl_age + fam_income + mat_ed + demo_ethn_v2 + genetic_af_african + genetic_af_european + genetic_af_east_asian + genetic_af_american".format(yvar, net), 
                                 re_formula="1", vc_formula={"fam_id": "0 + C(rel_family_id)",
                                                             "subjectkey": "0 + C(subjectkey)"},
                groups="mri_info_deviceserialnumber", data= data_both);
bothresults = both_mod.fit();
print(' ')
print('T-value: {}'.format(pd.DataFrame(bothresults.tvalues).loc['{}:genotype[T.2]'.format(net)]))
print(' ')
bothsummary = bothresults.summary();
print(bothsummary)

data_c = baseline_netdf[baseline_netdf['genotype'] == 1]
data_c[yvar] = data_c[yvar].astype(float)
data_c['mat_ed'] = data_c['mat_ed'].astype('category').cat.remove_unused_categories()
cc_mod = sm.MixedLM.from_formula("{}~ {} + pds + sex +cbcl_age + fam_income + mat_ed + demo_ethn_v2 + genetic_af_african + genetic_af_european + genetic_af_east_asian + genetic_af_american".format(yvar, net), 
                                 re_formula="1", vc_formula={"fam_id": "0 + C(rel_family_id)",
                                                             "subjectkey": "0 + C(subjectkey)"},
                groups="mri_info_deviceserialnumber", data= data_c);
ccresults = cc_mod.fit();
ccsummary = ccresults.summary();
print(' ')
print('T-value: {}'.format(pd.DataFrame(ccresults.tvalues).loc['{}'.format(net)]))
print(' ')
print(ccsummary)

data_a = baseline_netdf[baseline_netdf['genotype'] == 2]
data_a['mat_ed'] = data_a['mat_ed'].cat.remove_unused_categories()
data_a[yvar] = data_a[yvar].astype(float)
a_mod = sm.MixedLM.from_formula("{}~ {} + pds + sex +cbcl_age + fam_income + mat_ed + demo_ethn_v2 + genetic_af_african + genetic_af_european + genetic_af_east_asian + genetic_af_american".format(yvar, net), 
                                 re_formula="1", vc_formula={"fam_id": "0 + C(rel_family_id)",
                                                             "subjectkey": "0 + C(subjectkey)"},
                groups="mri_info_deviceserialnumber", data= data_a);
aresults = a_mod.fit();
asummary = aresults.summary();
print(' ')
print('T-value: {}'.format(pd.DataFrame(aresults.tvalues).loc['{}'.format(net)]))
print(' ')
print(asummary)

In [ ]:
plt.rcParams["font.family"] = "times new roman"
ax = sns.lmplot(x=net, y=yvar, 
           scatter = True, data=data_c, hue = 'Genotype',
           scatter_kws={"alpha":.4},
           palette=['#e75454'], line_kws={"alpha":1,"lw":3},
           y_jitter = True)
plt.ylabel('Anxiety Symptoms',  fontsize=24)
plt.xlabel('Connectivity', fontsize=24)
plt.xlim([-90, 10])
plt.ylim([0,16])
plt.rc('xtick', labelsize = 20) 
plt.rc('ytick', labelsize = 20) 
ax._legend.remove()
# plt.legend(fontsize = 19,
#            bbox_to_anchor= (0.5, 0.35, .75, 0.5),
#            title="Genotype", 
#            loc = 7,
#            title_fontsize = 24, 
#            shadow = True, 
#            facecolor = 'white');

plt.savefig(home + '/Drafts/Figures/Positive_Scatterplot.png',
           bbox_inches='tight', pad_inches = .1)


In [ ]:
ax = sns.lmplot(x=net, y=yvar, hue = 'Genotype',
           scatter = True, data=data_a,
           palette=['#5c78ec'],
           scatter_kws={"alpha":.4},
           line_kws={"alpha":1,"lw":3},
           y_jitter = True)
plt.ylabel('Anxiety Symptoms', fontsize=24)
plt.xlabel('Connectivity', fontsize=24)
plt.xlim([-90, 10])
plt.ylim([0,16])
plt.rc('xtick', labelsize = 20) 
plt.rc('ytick', labelsize = 20) 
ax._legend.remove()
# plt.legend(fontsize = 19,
#            bbox_to_anchor= (0.5, 0.35, .75, 0.5),
#            title="Genotype", 
#            loc = 7,
#            title_fontsize = 24, 
#            shadow = True, 
#            facecolor = 'white');

plt.savefig(home + '/Drafts/Figures/Negative_Scatterplot.png',
           bbox_inches='tight', pad_inches = .1)

In [ ]:
gen1 = baseline_netdf[baseline_netdf['Genotype'] == 'AA/AC']
gen2 = baseline_netdf[baseline_netdf['Genotype'] == 'CC']

net = 'net_pos_var'
ax = sns.lmplot(x=net, y=yvar, hue='Genotype', 
                scatter = False, data=baseline_netdf,
                fit_reg=True, y_jitter = True,
                palette=['#e75454', '#5c78ec'], 
                scatter_kws={"alpha":.4})
# sns.regplot(x=net, y=yvar, data=gen1, scatter=False,
#            line_kws={"alpha":1,"lw":3, "color":"#5c78ec"})
# sns.regplot(x=net, y=yvar, data=gen2, scatter=False,
#            line_kws={"alpha":1,"lw":3, "color":"#e75454"})

plt.ylabel('CBCL Anxiety Disorder Symptoms (raw scores)', fontsize = 13)
plt.xlabel('Connectivity of edges where A-allele carriers > non-carriers', fontsize = 13)
plt.rc('xtick', labelsize = 12) 
plt.rc('ytick', labelsize = 12) 
ax._legend.remove()
plt.legend(fontsize = 10,
           bbox_to_anchor= (0.4, 0.20, .85, 0.5),
           title="Genotype", 
           loc = 7,
           title_fontsize = 13, 
           shadow = True, 
           facecolor = 'white');
#plt.xlim([-90, 10])
#plt.ylim([0,16])
# plt.ylim([0,16])
plt.savefig(home + '/Drafts/Figures/Interaction_Scatterplot.png',
           bbox_inches='tight', pad_inches = .1, dpi=300)